In [8]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 mins 34 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_47457_7clstl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.471 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [11]:
import pandas as pd
import os
import numpy as np

In [9]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)

       average_income_per_grunnkrets  C_1.1.1.0  C_1.1.2.0  C_1.1.3.0  \
0                           561700.0          1          0          0   
1                           555720.0          1          0          0   
2                           495900.0          1          0          0   
3                           521240.0          1          0          0   
4                           585360.0          1          0          0   
...                              ...        ...        ...        ...   
12854                       505640.0          0          0          0   
12855                       396640.0          0          0          0   
12856                       549120.0          0          0          0   
12857                       498660.0          0          0          0   
12858                       579780.0          0          0          0   

       C_1.1.4.0  C_1.1.5.0  C_1.1.6.1  C_1.1.6.2  C_1.1.6.3  C_1.1.6.4  ...  \
0              0          0          0     

In [10]:
folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)

      average_income_per_grunnkrets  C_1.1.1.0  C_1.1.2.0  C_1.1.3.0  \
0                          551600.0          1          0          0   
1                          531240.0          1          0          0   
2                          523300.0          1          0          0   
3                          591380.0          1          0          0   
4                          594020.0          1          0          0   
...                             ...        ...        ...        ...   
8572                       560700.0          0          0          0   
8573                       593340.0          0          0          0   
8574                       629100.0          0          0          0   
8575                       590100.0          0          0          0   
8576                       529980.0          0          0          0   

      C_1.1.4.0  C_1.1.5.0  C_1.1.6.1  C_1.1.6.2  C_1.1.6.3  C_1.1.6.4  ...  \
0             0          0          0          0        

In [49]:
target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue']

#log transform
target_train = np.log1p(target_train)

target_train = [str(int(item*100000)) for item in target_train]

target_df = pd.DataFrame()
target_df["revenue"] = target_train

#for i in range(len(target_train)):
    #target_train[i] = int(target_train[i]*100000)

print(target_df)

      revenue
0      294433
1      321197
2      283901
3      233175
4      170982
...       ...
12854    8434
12855  103531
12856  366931
12857  153514
12858  120237

[12859 rows x 1 columns]


In [50]:
train_df = h2o.H2OFrame(features_train)
test_df = h2o.H2OFrame(features_test)
target = h2o.H2OFrame(target_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [32]:
h2o_automl = H2OAutoML(sort_metric='rmsle', max_runtime_secs=60, seed=666)

In [51]:
h2o_automl.train(x=test_df, y=target_df, training_frame=train_df)

H2OTypeError: Argument `y` should be an integer | string, got DataFrame       revenue
0      294433
1      321197
2      283901
3      233175
4      170982
...       ...
12854    8434
12855  103531
12856  366931
12857  153514
12858  120237

[12859 rows x 1 columns]

In [14]:
#Divide all on 100000
#lgbm_predictions = np.expm1(lgbm_predictions)